>**기출문제 제3회**
>>**01 작업형 제1유형**

## 옵션 설정

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) # 모든 칼럼이 출력되게 조절

In [2]:
import pandas as pd 

data = pd.read_csv('3_1.csv')
data

,city,homeType,latitude,longitude,garageSpaces,hasSpa,yearBuilt,numOfPatioAndPorchFeatures,lotSizeSqFt,avgSchoolRating,MedianStudentsPerTeacher,numOfBathrooms,numOfBedrooms,priceRange
0,austin,Single Family,30.197647,-97.816811,2,False,1981,0,9757.0,4.000000,13,NaN,3,350000-450000
1,austin,Single Family,30.329187,-97.752731,2,False,1951,0,10846.0,7.000000,16,NaN,4,650000+
2,austin,Single Family,30.313305,-97.699669,0,False,1954,0,9016.0,5.000000,15,NaN,3,350000-450000
3,austin,Single Family,30.156321,-97.748291,2,False,2008,0,5662.0,5.333333,15,NaN,3,0-250000
4,austin,Townhouse,30.366995,-97.753548,2,False,1984,0,3049.0,7.000000,16,NaN,2,350000-450000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,austin,Single Family,30.287403,-97.712151,0,False,1966,0,7797.0,6.666667,15,1.0,3,350000-450000
496,austin,Single Family,30.299253,-97.778496,0,False,2016,0,6011.0,7.000000,16,4.0,4,650000+
497,austin,Single Family,30.202967,-97.970039,0,False,2018,1,8712.0,7.000000,16,3.0,4,350000-450000
498,austin,Single Family,30.173611,-97.912315,0,False,2015,0,6621.0,9.000000,18,2.0,3,350000-450000


## 문제 1. 
- 다음 데이터에서 lotSizeSqFt이 큰 top 10을 구하고, top 10 값 중 가장 작은 값으로 해당 값을 대치하시오. 
- 또한, 건축 연도(yearBuilt)가 2000년도 이상인 IoTSizeSqFt의 평균을 구하시오.
- 소수점 첫째 자리에서 반올림, 대치된 IoTSizeSqFt 기준


In [3]:
# 코드 작성
lssqft_top10_data = data['lotSizeSqFt'].sort_values(ascending=False).head(10)
min_lssqft = float(lssqft_top10_data.min())
data.loc[data['lotSizeSqFt'] > min_lssqft, 'lotSizeSqFt'] = min_lssqft
data['lotSizeSqFt'].sort_values(ascending=False).head(10)

265    71438.4
399    71438.4
68     71438.4
175    71438.4
412    71438.4
97     71438.4
480    71438.4
440    71438.4
6      71438.4
71     71438.4
Name: lotSizeSqFt, dtype: float64

In [4]:
# 코드 작성
print(round(data.loc[data['yearBuilt'] >= 2000]['lotSizeSqFt'].mean()))

12955


## 문제 2. 
- 컬럼별 결측치 존재 여부를 확인하고, 결측치가 존재하는 경우 해당 컬럼의 중앙값으로 결측치를 대치하시오. 
- 결측치 대신 전과 후 표준편차 차이의 절댓값을 구하시오. 
- 소수점 둘째 자리까지 반올림


In [5]:
data2 = data.copy()

In [6]:
# 코드 작성
data2.isnull().sum()

city                           0
homeType                       0
latitude                       0
longitude                      0
garageSpaces                   0
hasSpa                         0
yearBuilt                      0
numOfPatioAndPorchFeatures     0
lotSizeSqFt                    0
avgSchoolRating                0
MedianStudentsPerTeacher       0
numOfBathrooms                11
numOfBedrooms                  0
priceRange                     0
dtype: int64

In [7]:
col_with_null = data2.columns[data2.isnull().any()].tolist()
col_with_null

['numOfBathrooms']

In [17]:
std_before = data2.loc[:, col_with_null].std()
print(std_before)

numOfBathrooms    0.989337
dtype: float64


In [24]:
# 코드 작성
# data_new = data2.fillna(data.loc[:, col_with_null].notnull().median())
# 에러이유
# data.loc[:, col_with_null].notnull() 은 True/False 값 시리즈가 되고,
# .median() 은 결국 True/False의 중앙값(1.0)으로 계산된다.
# -> 결측치를 1.0으로 채우는 문제 발생

# 정답 코드
median_value = data2[col_with_null].median()
data_new = data2.copy()
data_new[col_with_null] = \
    data_new[col_with_null].fillna(median_value)


In [26]:
std_after = data_new.loc[:, col_with_null].std()

std_diff = abs(std_after - std_before)
std_diff

numOfBathrooms    0.009786
dtype: float64

In [28]:
# 코드 작성
print("표준 편차 차이의 절대값: ")
print(np.round(float(std_diff.iloc[0]), 2))

표준 편차 차이의 절대값: 
0.01


## 문제 3. 
- 평균으로부터 1.5x 표준편차 범위 밖의 값을 이상치로 판단
- MedianStudentsPerTeacher의 이상치를 구하고, 이상치의 개수를 구하시오. 

In [10]:
# 코드 작성

